-----

Keyboard shortcuts:

| do this                  | by typing this   |
| ------------------------ |------------------|
| run cell                 | ⌃↩              |
| run cell and select next | ⇧↩             |

-----

In [1]:
from pprint import pprint
from string import Template

import urllib.request
import json
import csv
import codecs

In [2]:
class USZIPCodeRepository:
    CACHE = {}

    def __init__(self, data_url_prefix = 'https://raw.githubusercontent.com/yyu/GeoJSON-US/master'):
        self.data_url_prefix = data_url_prefix
        self.geojson_url_prefix = f'{data_url_prefix}/perZIPgeojson'

        self.refresh_zipcode_latlons(f'{data_url_prefix}/ZIPCodesGazetteer.tsv')
        self.refresh_available_zipcodes(f'{data_url_prefix}/perZIPgeojson/all_zipcodes.txt')


    def refresh_zipcode_latlons(self, url):
        lines = [ line.decode('UTF8').strip() for line in urllib.request.urlopen(url).readlines() ]
        tsv = csv.DictReader(lines, delimiter='\t')
        self.gazetteer = dict((d['GEOID'], {'lat': float(d['INTPTLAT']), 'lon': float(d['INTPTLONG'])}) for d in tsv)


    def refresh_available_zipcodes(self, url):
        lines = [ zipcode.decode('UTF8').strip() for zipcode in urllib.request.urlopen(url).readlines() ]
        self.zipcode_list = lines[1:] # ignore the first line
        self.zipcode_set = set(self.zipcode_list)


    def make_url(self, zipcode):
        return f'{self.data_url_prefix}/perZIPgeojson/{zipcode[0]}/{zipcode[1]}/{zipcode[2]}/{zipcode}.json'


    def fetch_zipcode(self, zipcode):
        '''returns a (dict, err) tuple where err could be a string for error message or None'''

        url = self.make_url(zipcode)

        if url in USZIPCodeRepository.CACHE:
            return (USZIPCodeRepository.CACHE[url], None)

        try:
            s = urllib.request.urlopen(url).read()
        except urllib.error.URLError as e:
            return (None, 'failed to get ' + url, ':', e.reason)

        j = json.loads(s)

        USZIPCodeRepository.CACHE[url] = j

        return (j, None)


    def fetch_zipcodes(self, *zipcodes):
        d = {"type": "FeatureCollection", "features": []}

        available_zipcodes = set(zipcodes) & self.zipcode_set

        for z in available_zipcodes:
            j, err = self.fetch_zipcode(z)

            if j is not None:
                d['features'].append(j)

        return d


In [3]:
import ipyleaflet as leaflet
import ipywidgets as widgets

class USMap:
    def __init__(self):
        self.us = USZIPCodeRepository()

        self.center = [47.621795, -122.334958]
        self.zoom = 8
        self.height = '500px'
        self.progress_bar_width = '500px'
        self.area_style = {'color':'#0000ff', 'weight': .5, 'fillColor':'#000077', 'fillOpacity':0.2}

        self.progress_bar = widgets.IntProgress(bar_style='info', layout=widgets.Layout(width=self.progress_bar_width))
        self.label = widgets.Label()
        self.progress_label = widgets.Label()
        self.info_box = widgets.HBox([self.progress_label, self.progress_bar])

        self.basemap = leaflet.basemaps.OpenMapSurfer.Roads
        self.basemap['name'] = 'basemap'
        self.heatmap_data = leaflet.basemaps.Strava.All
        self.heatmap_data['name'] = 'heatmap'
        self.heatmap = leaflet.basemap_to_tiles(self.heatmap_data)
        self.layers_control = leaflet.LayersControl()
        self.map_layout = widgets.Layout(height=self.height)

        self.map = None


    def enable_heatmap(self):
        self.map.add_layer(self.heatmap)


    def disable_heatmap(self):
        self.map.remove_layer(self.heatmap)


    def handle_interaction(self, **kwargs):
        '''mouse interaction handling'''
        if kwargs.get('type') == 'mousemove':
            self.label.value = str(kwargs.get('coordinates'))

    def fetch_zipcode(self, zipcode):
        d, err = self.us.fetch_zipcode(zipcode)
        if err is not None:
            print(err)
        return d


    def add_point(self, lat, lng):
        feature = {"type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [lng, lat]}}
        self.add_geojson(feature)


    def add_geojson(self, geojson, name='', popup=None):
        g = leaflet.GeoJSON(data=geojson, hover_style={'fillColor': '#00aaff'}, name=name)

        if popup is not None:
            g.popup = popup

        self.map += g


    def add_geojsons(self, geojsons, name=''):
        d = {"type": "FeatureCollection", "features": list(geojsons)}

        self.add_geojson(d, name)


    def add_zipcode(self, zipcode):
        d = self.fetch_zipcode(zipcode)
        if d is None:
            print('failed to add ' + zipcode + '.')
            return

        d['properties']['style'] = self.area_style

        text_template = Template('''<div>ZIP Code
                                        <ul class='list-group'>
                                            <li class='list-group-item'>$zipcode</li>
                                        </ul>
                                    </div>''')
        popup_text = text_template.substitute(zipcode=zipcode)
        popup = widgets.HTML(value=popup_text, placeholder='', description='')

        self.add_geojson(d, name=zipcode, popup=popup)


    def add_zipcodes(self, zipcodes):
        display(self.info_box)

        zipcodes = set(zipcodes)
        available_zipcodes = list(zipcodes & self.us.zipcode_set)
        available_zipcodes.sort()

        self.progress_bar.value = self.progress_bar.min
        self.progress_bar.max = len(available_zipcodes)

        for z in available_zipcodes:
            self.progress_label.value = z
            self.add_zipcode(z)
            self.progress_bar.value += 1
        self.progress_label.value = '%d/%d loaded' % (len(available_zipcodes), len(zipcodes))

        return available_zipcodes


    def display(self):
        if self.map is None:
            self.map = leaflet.Map( center=self.center, zoom=self.zoom, basemap=self.basemap, layout=self.map_layout)
            self.map.on_interaction(self.handle_interaction)
            self.map.add_control(self.layers_control)

        display(self.map)
        display(self.label)

In [4]:
m = USMap()
m.area_style = {'color':'#00ff00', 'weight': .5, 'fillColor':'#007700', 'fillOpacity':0.5}
m.zoom = 11

m.display()

Map(basemap={'url': 'http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 'max_zoom': 20, 'attribution': 'Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', 'name': 'basemap'}, center=[47.621795, -122.334958], controls=(LayersControl(),), layers=(TileLayer(attribution='Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', base=True, max_zoom=20, min_zoom=1, name='basemap', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size'], url='http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}'),), layout=Layout(height='500px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=11)

Label(value='')

In [5]:
m.add_zipcode('98109')

In [6]:
m.enable_heatmap()

In [7]:
#m.disable_heatmap()

-----

In [8]:
m2 = USMap()
m2.display()

Map(basemap={'url': 'http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 'max_zoom': 20, 'attribution': 'Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', 'name': 'basemap'}, center=[47.621795, -122.334958], controls=(LayersControl(),), layers=(TileLayer(attribution='Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', base=True, max_zoom=20, min_zoom=1, name='basemap', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size'], url='http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}'),), layout=Layout(height='500px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=8)

Label(value='')

In [9]:
added_zipcodes = m2.add_zipcodes('98%03d' % i for i in range(100))
print(added_zipcodes)

HBox(children=(Label(value=''), IntProgress(value=0, bar_style='info', layout=Layout(width='500px'))))

['98001', '98002', '98003', '98004', '98005', '98006', '98007', '98008', '98010', '98011', '98012', '98014', '98019', '98020', '98021', '98022', '98023', '98024', '98026', '98027', '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98036', '98037', '98038', '98039', '98040', '98042', '98043', '98045', '98047', '98050', '98051', '98052', '98053', '98055', '98056', '98057', '98058', '98059', '98065', '98068', '98070', '98072', '98074', '98075', '98077', '98087', '98092']


-----

In [10]:
m3 = USMap()
m3.zoom = 11
m3.display()

Map(basemap={'url': 'http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 'max_zoom': 20, 'attribution': 'Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', 'name': 'basemap'}, center=[47.621795, -122.334958], controls=(LayersControl(),), layers=(TileLayer(attribution='Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', base=True, max_zoom=20, min_zoom=1, name='basemap', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size'], url='http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}'),), layout=Layout(height='500px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=11)

Label(value='')

In [11]:
m3.add_zipcodes(['98109', '98121', '98040'])

HBox(children=(Label(value=''), IntProgress(value=0, bar_style='info', layout=Layout(width='500px'))))

['98040', '98109', '98121']

-----

In [12]:
m4 = USMap()
m4.display()

Map(basemap={'url': 'http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 'max_zoom': 20, 'attribution': 'Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', 'name': 'basemap'}, center=[47.621795, -122.334958], controls=(LayersControl(),), layers=(TileLayer(attribution='Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', base=True, max_zoom=20, min_zoom=1, name='basemap', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size'], url='http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}'),), layout=Layout(height='500px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=8)

Label(value='')

In [13]:
added_zipcodes = m4.add_zipcodes('98%03d' % i for i in range(100))
print(added_zipcodes)

HBox(children=(Label(value=''), IntProgress(value=0, bar_style='info', layout=Layout(width='500px'))))

['98001', '98002', '98003', '98004', '98005', '98006', '98007', '98008', '98010', '98011', '98012', '98014', '98019', '98020', '98021', '98022', '98023', '98024', '98026', '98027', '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98036', '98037', '98038', '98039', '98040', '98042', '98043', '98045', '98047', '98050', '98051', '98052', '98053', '98055', '98056', '98057', '98058', '98059', '98065', '98068', '98070', '98072', '98074', '98075', '98077', '98087', '98092']


-----

In [14]:
m5 = USMap()
marker90033 = m5.us.gazetteer['90033']
m5.center = [marker90033['lat'], marker90033['lon']]
m5.display()

Map(basemap={'url': 'http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 'max_zoom': 20, 'attribution': 'Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', 'name': 'basemap'}, center=[34.05038, -118.211991], controls=(LayersControl(),), layers=(TileLayer(attribution='Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', base=True, max_zoom=20, min_zoom=1, name='basemap', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size'], url='http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}'),), layout=Layout(height='500px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=8)

Label(value='')

In [15]:
added_zipcodes = m5.add_zipcodes('9%04d' % i for i in range(1000))
print(added_zipcodes)

HBox(children=(Label(value=''), IntProgress(value=0, bar_style='info', layout=Layout(width='500px'))))

['90001', '90002', '90003', '90004', '90005', '90006', '90007', '90008', '90010', '90011', '90012', '90013', '90014', '90015', '90016', '90017', '90018', '90019', '90020', '90021', '90022', '90023', '90024', '90025', '90026', '90027', '90028', '90029', '90031', '90032', '90033', '90034', '90035', '90036', '90037', '90038', '90039', '90040', '90041', '90042', '90043', '90044', '90045', '90046', '90047', '90048', '90049', '90056', '90057', '90058', '90059', '90061', '90062', '90063', '90064', '90065', '90066', '90067', '90068', '90069', '90071', '90073', '90077', '90079', '90089', '90090', '90094', '90095', '90201', '90210', '90211', '90212', '90220', '90221', '90222', '90230', '90232', '90240', '90241', '90242', '90245', '90247', '90248', '90249', '90250', '90254', '90255', '90260', '90262', '90263', '90265', '90266', '90270', '90272', '90274', '90275', '90277', '90278', '90280', '90290', '90291', '90292', '90293', '90301', '90302', '90303', '90304', '90305', '90401', '90402', '90403', 

-----